In [ ]:
#### Dans ce programme ####
# On va cherche pour une région à faire le classement des meilleurs investissements immobilier depuis 2019 #

In [75]:
# Fonctions utiles
def dlldvfdepartement(departement_name): # Fonction pour telecharger le dvf du département selectionné en figdep
    codedepartement = liste_departement.loc[liste_departement["nom_departement"]==departement_name,"code"].value_counts().keys()[0]
    try:
        dvf = pd.read_csv(f"https://github.com/enzofranzy/DVF-dash-plotly/blob/main/dvf_id_total/{codedepartement}ret_id_total.csv.xz?raw=true", compression='xz') # Telechargement du dvf directement sur GITHUB en .csv.xz
        dvf['code_departement'] = dvf['code_departement'].astype('str')
        dvf['code_commune'] = dvf['code_commune'].astype('str')
        dvf['code_departement'] = dvf['code_departement'].str.zfill(2) # nous voulons que chaque code de département ait deux caractères, donc nous allons utiliser zfill(2) pour ajouter un "0" devant les valeurs qui n'en ont qu'un
        dvf['code_commune'] = dvf['code_commune'].str.zfill(5)
        dvf["code_commune_fig_commune"] = dvf['code_commune'] # Création de la colonne code_commune_fig_commune
        dvf["section"] = dvf["id_parcelle"].str.slice(0, 10) # Corrige la colonne section car erreur fichier source => Correction faite sur Download_DVFbyRegion => lignes inutiles
        dvf["Nombre_Dépendance"] = dvf["Nombre_Dépendance"].fillna(0) #Corrige la colonne Nombre_dépendance pour éviter les erreurs ensuite
        dvf["Nombre_Maison"] = dvf["Nombre_Maison"].fillna(0) #Corrige la colonne Nombre_dépendance pour éviter les erreurs ensuite
        dvf["Nombre_Appartement"] = dvf["Nombre_Appartement"].fillna(0) #Corrige la colonne Nombre_dépendance pour éviter les erreurs ensuite
        dvf["Nombre_Local industriel. commercial ou assimilé"] = dvf["Nombre_Local industriel. commercial ou assimilé"].fillna(0) #Corrige la colonne Nombre_dépendance pour éviter les erreurs ensuite
        # Traitement des Villes à arrondissements
        if codedepartement == "13":
            dvf['nom_commune'] = dvf['nom_commune'].str.replace(r'Marseille (1er|2e|3e|4e|5e|6e|7e|8e|9e|10e|11e|12e|13e|14e|15e|16e) Arrondissement', 'Marseille', regex=True)
            L = ["13201","13202","13203","13204","13205","13206","13207","13208","13209","13210","13211","13212","13213","13214","13215","13216"] # Arrondissements Marseille 
            dvf["code_commune_fig_commune"] = dvf["code_commune"].replace(L,"13055")
        elif codedepartement == "69":
            dvf['nom_commune'] = dvf['nom_commune'].str.replace(r'Lyon (1er|2e|3e|4e|5e|6e|7e|8e|9e) Arrondissement', 'Lyon', regex=True)
            L = ["69381","69382","69383","69384","69385","69386","69387","69388","69389"] # Lyon
            dvf["code_commune_fig_commune"] = dvf["code_commune"].replace(L,"69123")
        elif codedepartement == "75":
            dvf['nom_commune'] = dvf['nom_commune'].str.replace(r'Paris (1er|2e|3e|4e|5e|6e|7e|8e|9e|10e|11e|12e|13e|14e|15e|16e|17e|18e|19e|20e) Arrondissement', 'Paris', regex=True)
            L = ["75101","75102","75103","75104","75105","75106","75107","75108","75109","75110","75111","75112","75113","75114","75115","75116","75117","75118","75119","75120"]
            dvf["code_commune_fig_commune"] = dvf["code_commune"].replace(L,"75056")
    except:
        print(f"Il est impossible de télécharger le dvf pour ce departement {departement_name}")
        dvf = ""
    return dvf

In [76]:
# Importations des modules (library)
import pandas as pd # pip install pandas
import gc
import requests # pip install requests
import json

# Telechargement de liste_departement
liste_departement = pd.read_csv("https://raw.githubusercontent.com/enzofranzy/DVF-dash-plotly/main/departements-france.csv").drop(columns=["Unnamed: 0"]).rename(columns={"code_departement":"code"})

In [77]:
# Affiche la liste des nom_région de France
liste_departement["nom_region"].unique()

array(['Auvergne-Rhône-Alpes', 'Hauts-de-France',
       "Provence-Alpes-Côte d'Azur", 'Grand Est', 'Occitanie',
       'Normandie', 'Nouvelle-Aquitaine', 'Centre-Val de Loire',
       'Bourgogne-Franche-Comté', 'Bretagne', 'Corse', 'Pays de la Loire',
       'Île-de-France'], dtype=object)

In [78]:
# Définit la région étudiée
region = "Provence-Alpes-Côte d'Azur"
list_dep = list(liste_departement[liste_departement["nom_region"]==region]["nom_departement"].unique()) # liste les codes departements de la région

In [79]:
# Programme compilant les datas dans une variable dvfglobal
for i in range (0,len(list_dep)):
    dvf = dlldvfdepartement(list_dep[i])
    if i==0:
        dvfglobal = dvf
    else:
        dvfglobal = pd.concat([dvfglobal,dvf])

In [80]:
# on cherche uniquement les appartements
dvfglobal["annee"] = dvfglobal["annee"].astype('str')
dvfglobal = dvfglobal[dvfglobal["Nombre_Appartement"]>0]
dvfglobal = dvfglobal[(dvfglobal["Nombre_Local industriel. commercial ou assimilé"]==0) & (dvfglobal["Nombre_Maison"]==0)]

In [81]:
# Construction des tables pour affichage
tcd_typelocal = pd.pivot_table(dvfglobal, values='prixM2',index=['nom_commune','code_commune_fig_commune'],columns='annee',aggfunc={'prixM2': 'median'}, fill_value=0).reset_index()
tcd_typelocal["prixM2_moyen"] = (tcd_typelocal["2019"] + tcd_typelocal["2020"] + tcd_typelocal["2021"] + tcd_typelocal["2022"] + tcd_typelocal["2023"])/5
tcd_typelocal1 = pd.pivot_table(dvfglobal, values='id_mutation',index=['nom_commune','code_commune_fig_commune'],columns='annee',aggfunc={'id_mutation': 'count'}, fill_value=0).reset_index()
tcd_typelocal1["Nb biens vendus"] = (tcd_typelocal1["2019"] + tcd_typelocal1["2020"] + tcd_typelocal1["2021"] + tcd_typelocal1["2022"] + tcd_typelocal1["2023"])
tcd_typelocal = pd.merge(tcd_typelocal,tcd_typelocal1,on=['nom_commune','code_commune_fig_commune'],how='left')

# filtre sur le nombre de biens vendus pour avoir un minimum de certitudes sur les données
tcd_typelocal = tcd_typelocal[tcd_typelocal["Nb biens vendus"]>100]
tcd_typelocal["% Evolution depuis 2019"] = 100*(tcd_typelocal["2023_x"]-tcd_typelocal["2019_x"])/tcd_typelocal["2023_x"]
tcd_typelocal["% Evolution entre 2022 & 2023"] = 100*(tcd_typelocal["2023_x"]-tcd_typelocal["2022_x"])/tcd_typelocal["2022_x"]
del tcd_typelocal1 ; gc.collect()

141

In [82]:
# filtre 
df_ = tcd_typelocal.sort_values("% Evolution depuis 2019",ascending=False).reset_index().drop(columns=["index"]) # permet de selectionner les villes de grande taille au moins x ventes par ville
df_.to_excel(f"EvolutionprixM2depuis2019{region}.xlsx")